In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from datetime import date
from datetime import timedelta 

from pathlib import Path
import requests
import json
import os
from dotenv import load_dotenv

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from string import punctuation
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
import re

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Set the path for the csv files

data_csv = Path("./data/TSLA_yr2019News_FullArtciles_df.csv")

# Reading csv files

df = pd.read_csv(data_csv)

df

,Unnamed: 0,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers,full_text
0,0,https://www.fool.com/investing/2019/12/31/why-...,https://cdn.snapi.dev/images/v1/0/e/etf13-35.jpg,Why the Stock Market Soured on Tesla and Centu...,The carmaker's CEO had some Boring news to rep...,The Motley Fool,31 Dec 2019,[],Negative,Article,"['CTL', 'TSLA']",Many of us are preparing for the new year and ...
1,1,https://www.zacks.com/stock/news/694580/tesla-...,https://cdn.snapi.dev/images/v1/i/l/0x600-39.jpg,Tesla to Start Delivery of Model 3 Built at Ch...,The construction of Gigafactory3 in Shanghai i...,Zacks Investment Research,30 Dec 2019,[],Neutral,Article,['TSLA'],"Tesla, Inc. (TSLA - Free Report) will start de..."
2,2,https://www.zacks.com/stock/news/694510/tesla-...,https://cdn.snapi.dev/images/v1/b/i/tesla-1724...,Tesla (TSLA) Rolls Out First Model 3s Built in...,Tesla Motors (TSLA) remains near the peak of i...,Zacks Investment Research,30 Dec 2019,[],Positive,Article,['TSLA'],"Monday, December 30, 2019\nAdvance Trade in U...."
3,3,https://www.fool.com/investing/2019/12/29/how-...,https://cdn.snapi.dev/images/v1/e/t/105924582-...,How Many Vehicles Will Tesla Deliver in Q4?,The company is slated to report its quarterly ...,The Motley Fool,29 Dec 2019,['earnings'],Neutral,Article,['TSLA'],Sometime this week -- probably on Thursday or ...
4,4,https://www.fool.com/investing/2019/12/28/why-...,https://cdn.snapi.dev/images/v1/3/c/urlhttps3a...,Why Your 2020 Resolution Should Be to Buy More...,"After a great 2019 capstone to the 2010s, tech...",The Motley Fool,28 Dec 2019,[],Positive,Article,"['AMZN', 'GOOG', 'GOOGL', 'MSFT', 'NFLX', 'TSL...",After a decade of virtually no returns -- than...
...,...,...,...,...,...,...,...,...,...,...,...,...
285,285,https://www.fool.com/investing/2019/11/16/what...,https://cdn.snapi.dev/images/v1/f/r/fredw22-1.jpg,What We Know About Tesla's Pickup Truck,The electric-car company will finally unveil t...,The Motley Fool,16 Nov 2019,[],Neutral,Article,['TSLA'],As if electric-car maker Tesla (NASDAQ:TSLA) i...
286,286,https://www.fool.com/investing/2019/11/15/cons...,https://cdn.snapi.dev/images/v1/j/y/105797044-...,"""Consumer Reports"" Now Recommends Tesla's Mode...",The news comes as Tesla aims for record delive...,The Motley Fool,15 Nov 2019,[],Positive,Article,['TSLA'],In a reversal of a previous decision not to re...
287,287,https://www.fool.com/investing/2019/11/15/3-st...,https://cdn.snapi.dev/images/v1/n/r/0101ee.jpg,3 Stocks That Soared This Earnings Season,Don't miss the stories behind these companies'...,The Motley Fool,15 Nov 2019,['earnings'],Positive,Article,"['AAPL', 'DIS', 'TSLA']","As usual, earnings season brought with it its ..."
288,288,https://www.zacks.com/stock/news/619555/tesla-...,https://cdn.snapi.dev/images/v1/o/j/tes723-5.jpg,Tesla (TSLA) to Open First European Gigafactor...,Car production in Tesla's (TSLA) Berlin Gigafa...,Zacks Investment Research,14 Nov 2019,[],Neutral,Article,['TSLA'],Tesla ’s (TSLA - Free Report) electric revolut...


# Sentiment based on NLTK Vader

In [3]:
# Create function that computes average compound sentiment for each day

def vader_sentiment_summarizer(df_column):

    vader_analyzer = SentimentIntensityAnalyzer()
    
    vader_sentiments = []
    for text in df_column:
        
        sentiment = vader_analyzer.polarity_scores(text)
        
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        vader_sentiments.append({
            "Compound": compound,
            "Positive": pos,
            "Neutral": neu,
            "Negative": neg,
            "Text": text})
        
    vader_sentiments_df = pd.DataFrame.from_dict(vader_sentiments)

    return vader_sentiments_df

In [4]:
vader_df = vader_sentiment_summarizer(df["full_text"])
vader_df

,Compound,Positive,Neutral,Negative,Text
0,0.9109,0.064,0.885,0.051,Many of us are preparing for the new year and ...
1,0.9961,0.129,0.858,0.014,"Tesla, Inc. (TSLA - Free Report) will start de..."
2,-0.8271,0.085,0.810,0.105,"Monday, December 30, 2019\nAdvance Trade in U...."
3,0.9880,0.090,0.897,0.013,Sometime this week -- probably on Thursday or ...
4,0.9934,0.105,0.837,0.058,After a decade of virtually no returns -- than...
...,...,...,...,...,...
285,0.9811,0.060,0.940,0.000,As if electric-car maker Tesla (NASDAQ:TSLA) i...
286,0.9850,0.107,0.866,0.027,In a reversal of a previous decision not to re...
287,0.9962,0.169,0.827,0.004,"As usual, earnings season brought with it its ..."
288,0.9947,0.110,0.864,0.026,Tesla ’s (TSLA - Free Report) electric revolut...


In [15]:
#vader_df.to_csv("./data/vader_sentiments.csv", header=vader_df.columns)

# Sentiment based on TextBlob with LEMMATIZER

In [75]:
""" Create the Tokenizer function with LEMMATIZER """

def text_blob_lemmatizer(df_column):
    
    wnl = WordNetLemmatizer()
    
    # Set stop words
    stop = stopwords.words('english')
    stop.append("u")
    stop.append("it'")
    stop.append("'s")
    stop.append("n't")
    stop.append("…")
    stop.append("\`")
    stop.append("``")
    stop.append("char")
    stop.append("''")
    stop.append("'")
    stop.append("/")
    stop.append("r")
    stop.append(" ")
    stop.append("chars")
    stop.append("...")
    stop = set(stop)
    
    new_column = df_column.apply(lambda x: " ".join(x.lower() for x in x.split()))
    new_column = new_column.str.replace("[^a-zA-Z0-9]"," ")
    new_column = new_column.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    new_column = new_column.apply(lambda x: " ".join([wnl.lemmatize(word) for word in x.split()]))
    
    return new_column

In [76]:
textblob_lemmatized = pd.DataFrame(text_blob_lemmatizer(df["full_text"]))
textblob_lemmatized.full_text

0      many u preparing new year change hope bring ma...
1      tesla inc tsla free report start delivering mo...
2      monday december 30 2019 advance trade good mon...
3      sometime week probably thursday friday tesla n...
4      decade virtually return thanks large part burs...
                             ...                        
285    electric car maker tesla nasdaq tsla already p...
286    reversal previous decision recommend tesla nas...
287    usual earnings season brought fair share surpr...
288    tesla tsla free report electric revolution con...
289    tesla nasdaq tsla bear reason worry green car ...
Name: full_text, Length: 290, dtype: object

In [77]:
# Create the textblob sentiment function

def textblob_sentiment_summarizer(df_column):
    polarity = TextBlob(df_column).sentiment.polarity
    return polarity

In [80]:
textblob_lemmatized_polarity = textblob_lemmatized.full_text.apply(textblob_sentiment_summarizer)

textblob_lemmatized_polarity

0      0.087333
1      0.028692
2      0.078244
3      0.119706
4      0.100860
         ...   
285    0.075481
286    0.193791
287    0.174148
288    0.103478
289    0.104454
Name: full_text, Length: 290, dtype: float64

In [81]:
#textblob_lemmatized_polarity.to_csv("./data/textblob_lemmatized_sentiments.csv")

# Sentiment based on TextBlob with STEMMER

In [82]:
""" Create the Tokenizer function with STEMMER """

def text_blob_stemmer(df_column):
    
    st = PorterStemmer()
    
    # Set stop words
    stop = stopwords.words('english')
    stop.append("u")
    stop.append("it'")
    stop.append("'s")
    stop.append("n't")
    stop.append("…")
    stop.append("\`")
    stop.append("``")
    stop.append("char")
    stop.append("''")
    stop.append("'")
    stop.append("/")
    stop.append("r")
    stop.append(" ")
    stop.append("chars")
    stop.append("...")
    stop = set(stop)
    
    new_column = df_column.apply(lambda x: " ".join(x.lower() for x in x.split()))
    new_column = new_column.str.replace("[^a-zA-Z0-9]"," ")
    new_column = new_column.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    new_column = new_column.apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
    
    return new_column

In [83]:
textblob_stemmed = pd.DataFrame(text_blob_stemmer(df["full_text"]))
textblob_stemmed.full_text

0      mani us prepar new year chang hope bring mani ...
1      tesla inc tsla free report start deliv model 3...
2      monday decemb 30 2019 advanc trade good month ...
3      sometim week probabl thursday friday tesla nas...
4      decad virtual return thank larg part burst tec...
                             ...                        
285    electr car maker tesla nasdaq tsla alreadi pul...
286    revers previou decis recommend tesla nasdaq ts...
287    usual earn season brought fair share surpris v...
288    tesla tsla free report electr revolut continu ...
289    tesla nasdaq tsla bear reason worri green car ...
Name: full_text, Length: 290, dtype: object

In [86]:
textblob_stemmed_polarity = textblob_stemmed.full_text.apply(textblob_sentiment_summarizer)

textblob_stemmed_polarity

0      0.112693
1     -0.029774
2      0.099321
3      0.023636
4      0.057399
         ...   
285    0.091288
286    0.028636
287    0.159366
288    0.091447
289    0.061764
Name: full_text, Length: 290, dtype: float64

In [87]:
#textblob_stemmed_polarity.to_csv("./data/textblob_stemmed_sentiments.csv")